#### Imports

In [1]:
# imports
%matplotlib inline
%pylab inline


import numpy as np
import matplotlib.pyplot as plt
import scipy
import copy
import random

# Dim reduction tools
import umap
from sklearn.linear_model import LinearRegression
import torch
import torch.utils.data
from torch import optim
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import TensorDataset
from sklearn.model_selection import ShuffleSplit

# 
from utils import *
import segypy 
import pickle

#widget imports
from ipywidgets import interact, VBox, HBox
import ipywidgets as widgets

Populating the interactive namespace from numpy and matplotlib


#### Copy paste "utils.py" because it wont load/import:

In [2]:
# utils
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

def load_seismic(filename, inlines=[1300, 1502, 2], xlines=[1500, 2002, 2]):
    inl = np.arange(*inlines)
    crl = np.arange(*xlines)
    seis, header, trace_headers = segypy.readSegy(filename)
    amplitude = seis.reshape(header['ns'], inl.size, crl.size)
    lagtime = trace_headers['LagTimeA'][0]*-1
    twt = np.arange(lagtime, header['dt']/1e3*header['ns']+lagtime, header['dt']/1e3)
    return amplitude, twt

def load_horizon(filename, inlines=[1300, 1502, 2], xlines=[1500, 2002, 2]):
    inl = np.arange(*inlines)
    crl = np.arange(*xlines)
    hrz = np.recfromtxt(filename, names=['il','xl','z'])
    horizon = np.zeros((len(inl), len(crl)))
    for i, idx in enumerate(inl):
        for j, xdx in enumerate(crl):
            time = hrz['z'][np.where((hrz['il']== idx) & (hrz['xl'] == xdx))]
            if len(time) == 1:
                horizon[i, j] = time 

    return horizon

def colorbar(mappable):
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax)

def interpolate_horizon(horizon):
    points = []
    wanted = []
    for i in range(horizon.shape[0]):
        for j in range(horizon.shape[1]):
            if horizon[i, j] != 0.:
                points.append([i, j, horizon[i, j]])
            else:
                wanted.append([i, j])
    
    points = np.array(points)
    zs2 = scipy.interpolate.griddata(points[:, 0:2], points[:, 2], wanted, method="cubic")
    for p, val in zip(wanted, zs2):
        horizon[p[0], p[1]] = val
    
    return horizon

def plot_section_horizon_and_well(ax, amplitude, horizon, twt, inline=38, well_pos=276//2):
    hrz_idx = [np.abs(twt-val).argmin() for val in horizon[inline, :]]
    
    h_bin = np.zeros((amplitude.shape[0], amplitude.shape[2]))
    for i, val in enumerate(hrz_idx):
        h_bin[val, i] = 1

    clip = abs(np.percentile(amplitude, 0.8))
    ax.imshow(amplitude[:, inline], cmap="Greys", vmin=-clip, vmax=clip)
    ax.plot(range(len(hrz_idx)), hrz_idx, linewidth=5, color="black")
    ax.axvline(well_pos, color="red", linewidth=5)

def flatten_on_horizon(amplitude, horizon, twt, top_add=12, below_add=52):
    traces = np.zeros((horizon.shape[0], horizon.shape[1], top_add+below_add))
    for i in range(horizon.shape[0]):
        hrz_idx = [np.abs(twt-val).argmin() for val in horizon[i, :]]
        for j in range(horizon.shape[1]):
            traces[i, j, :] = amplitude[hrz_idx[j]-top_add:hrz_idx[j]+below_add, i, j]

    return traces

class VAE(nn.Module):
    def __init__(self, hidden_size):
        super(VAE, self).__init__()

        # Encoder
        self.conv1 = nn.Conv1d(2, 3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(3, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv1d(32, 32, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(256, 128)

        # Latent space
        self.fc21 = nn.Linear(128, hidden_size)
        self.fc22 = nn.Linear(128, hidden_size)

        # Decoder
        self.fc3 = nn.Linear(hidden_size, 128)
        self.fc4 = nn.Linear(128, 256)
        self.deconv1 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose1d(32, 32, kernel_size=4, stride=2, padding=1)
        self.conv5 = nn.Conv1d(32, 2, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        
        out = out.view(out.size(0), -1)
        h1 = self.relu(self.fc1(out))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            if mu.is_cuda:
                eps = eps.cuda()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))

        out = self.relu(self.fc4(h3))

        out = out.view(out.size(0), 32, 8)
        out = self.relu(self.deconv1(out))
        out = self.relu(self.deconv2(out))
        out = self.relu(self.deconv3(out))
        out = self.conv5(out)
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, z
    
def loss_function(recon_x, x, mu, logvar, window_size=64):
    criterion_mse = nn.MSELoss(size_average=False)
    MSE = criterion_mse(recon_x.view(-1, 2, window_size), x.view(-1, 2, window_size))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + KLD    
    
# Function to perform one epoch of training
def train(epoch, model, optimizer, train_loader, cuda=False, log_interval=10):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
#         print(data.shape)

        if cuda:
            data = data.cuda()

        optimizer.zero_grad()
        recon_batch, mu, logvar, _ = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item() * data.size(0)
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() * data.size(0) / len(train_loader.dataset)))

    train_loss /= len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss))
    return train_loss


# Function to perform evaluation of data on the model, used for testing
def test(epoch, model, test_loader, cuda=False, log_interval=10):
    model.eval()
    test_loss = 0
    with torch.set_grad_enabled(False):
        for i, (data, _) in enumerate(test_loader):
            if cuda:
                data = data.cuda()
            data = Variable(data)
            recon_batch, mu, logvar, _ = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item() * data.size(0)

        test_loss /= len(test_loader.dataset)
        print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss


# Function to forward_propagate a set of tensors and receive back latent variables and reconstructions
def forward_all(model, all_loader, cuda=False):
    model.eval()
    reconstructions, latents = [], []
    with torch.set_grad_enabled(False):
        for i, (data, _) in enumerate(all_loader):
            if cuda:
                data = data.cuda()
            data = Variable(data)
            recon_batch, mu, logvar, z = model(data)
            reconstructions.append(recon_batch.cpu())
            latents.append(z.cpu())
    return torch.cat(reconstructions, 0), torch.cat(latents, 0)

In [3]:
# testing VAE architecture

# x = torch.randn((100, 2, 64))
# print("init size:", x.size())

# model = VAE(hidden_size=8)
# y = model(x)
# # print(y)

# print("output size", y)

# 1. 'BACKEND':

### 1.1 DataHolder + Processor

In [4]:
class DataHolder:
    def __init__(self, field_name, inlines, xlines):
        
        # User input attributes
        self.field_name = field_name
        self.inlines = inlines
        self.xlines = xlines

        # KEY data for processing
        self.near = None
        self.far = None
        self.twt = None
        self.horizon = None
        
        # Dictionaries for multiple possible entries
        self.horizons = {}
        self.wells = {}
        
    def add_near(self, fname):
        self.near, twt = load_seismic(fname, inlines=self.inlines, xlines=self.xlines)
        self.twt = twt

    def add_far(self, fname):
        self.far, twt = load_seismic(fname, inlines=self.inlines, xlines=self.xlines)
        assert (self.twt == twt).all, "This twt does not match the twt from the previous segy"
        
    def add_horizon(self, fname):
        self.horizon = interpolate_horizon(load_horizon(fname, inlines=self.inlines, xlines=self.xlines))
        
    def add_well(self, well_id, well_i, well_x):
        self.wells[well_id] = [well_i, well_x]
           
            
class Processor:
    def __init__(self, Data):
        self.raw = [Data.near, Data.far]
        self.twt = Data.twt
        self.out = None
        self.horizon = Data.horizon
    
    def flatten(self, data, top_add=12, below_add=52):
        out = []
        print('before', data[0].shape)
        
        for amplitude in data:
            traces = np.zeros((self.horizon.shape[0], self.horizon.shape[1], top_add+below_add))
            for i in range(self.horizon.shape[0]):
                hrz_idx = [np.abs(self.twt-val).argmin() for val in self.horizon[i, :]] 
                for j in range(self.horizon.shape[1]):
                    traces[i, j, :] = amplitude[hrz_idx[j]-top_add:hrz_idx[j]+below_add, i, j]
            out.append(traces)
        print('after', out[0].shape)
        return out
    
    def normalise(self, data):
        well_i=38
        well_x=138
        out = []
        for i in data:
            well_variance = np.mean(np.std(i[well_i - 2:well_i + 1, well_x - 2:well_x + 1], 2))
            i /= well_variance
            out.append(i)

        return out

    def to_2d(self, data):
        return [i.reshape(-1, data[0].shape[-1]) for i in data]
        
    def average_neighbours(self, neighbours=10):
        return 'not implemented yet'
          
    def stack_traces(self, data):
        return np.concatenate([i for i in data], 1)
    
    def run_AVO(self):
#         print(self.out[0].shape, self.out[1].shape)
        x_avo = self.out[0]
        y_avo = self.out[1] - self.out[0]

        lin_reg = LinearRegression(fit_intercept=False, normalize=False, copy_X=True, n_jobs=1)
        lin_reg.fit(x_avo.reshape(-1, 1), y_avo.reshape(-1, 1))

        print("Linear Regression coefficient: %1.2f" % lin_reg.coef_[0, 0])
        return y_avo - lin_reg.coef_ * x_avo
    
    def __call__(self, flatten=False, normalise=False, label='FF'):
        self.out = copy.copy(self.raw)
        
        if flatten[0]:
            self.out = self.flatten(self.out, flatten[1], flatten[2])
        else: # perform the same axis manipulation as flattening method
            self.out[0] = self.out[0].reshape([self.out[0].shape[1], self.out[0].shape[2], self.out[0].shape[0]])
            self.out[1] = self.out[1].reshape([self.out[1].shape[1], self.out[1].shape[2], self.out[1].shape[0]])
     
        if normalise:
            self.out = self.normalise(self.out)
        
        # arrays from 3d to 2d
        self.out = self.to_2d(self.out)
        
        # Find fluid factor
        self.FF = self.run_AVO()
            
        # Stack the traces for output
        self.out = self.stack_traces(self.out)
        
        return [self.out, self.FF] 

### 1.2 ModelAgent -> UMAP + VAE + ...

In [5]:
class ModelAgent:
    def __init__(self, data):
        self.input = data[0]
        self.FF = data[1]
        self.embedding = None
        print("ModelAgent initialised")
        
    def plot_2d(self, data, label):
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        sc = ax.scatter(data[:, 0], data[:, 1], s=2.0, c=np.min(label, 1))
        return 1
        
    def plot_3d(self, data, feature):
        return 'Not implemented'
        
        
class UMAP(ModelAgent):
    def __init__(self, data):
        super().__init__(data)

    
    def reduce(self, n_neighbors = 50, min_dist=0.001):
        embedding = umap.UMAP(n_neighbors=n_neighbors,
                      min_dist=min_dist,
                      metric='correlation', 
                               verbose=True,
                            random_state=42).fit_transform(self.input)
        self.embedding = embedding        
        return embedding
        
    
class VAE_model(ModelAgent):
    def __init__(self, data):
        super().__init__(data)
    
    def create_dataloader(self, batch_size=32):
        # split the concatenated input back into two arrays
        X = torch.from_numpy(np.stack(np.split(self.input, 2, axis=1), 1)).float()
        
        # Create a stacked representation and a zero tensor so we can use the standard Pytorch TensorDataset
        y = torch.from_numpy(np.zeros((X.shape[0], 1))).float()
        
        split = ShuffleSplit(n_splits=1, test_size=0.5)
        for train_index, test_index in split.split(X):
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]

        train_dset = TensorDataset(X_train, y_train)
        test_dset = TensorDataset(X_test, y_test)
        all_dset = TensorDataset(X, y)

        kwargs = {'num_workers': 1, 'pin_memory': True}
        self.train_loader = torch.utils.data.DataLoader(train_dset, batch_size=batch_size, shuffle=True, **kwargs)
        self.test_loader = torch.utils.data.DataLoader(test_dset, batch_size=batch_size, shuffle=False, **kwargs)
        self.all_loader = torch.utils.data.DataLoader(all_dset, batch_size=batch_size, shuffle=False, **kwargs)
        
    def train_vae(self, cuda=False, epochs=5, hidden_size=8):
        set_seed(42)  # Set the random seed
        self.model = VAE(hidden_size)  # Inititalize the model

        # use cuda if chosen
        if cuda:
            self.model.cuda()

        # Create a gradient descent optimizer
        optimizer = optim.Adam(self.model.parameters(), lr=1e-2, betas=(0.9, 0.999))

        # Store and plot losses
        self.losses = []

        # Start training loop
        for epoch in range(1, epochs + 1):
            tl = train(epoch, self.model, optimizer, self.train_loader, cuda=False)  # Train model on train dataset
            testl = test(epoch, self.model, self.test_loader, cuda=False)  # Validate model on test dataset
            self.losses.append([tl, testl])
        
    def run_vae(self):
        _, self.zs = forward_all(self.model, self.all_loader, cuda=False)
        
    def vae_umap(self):
        transformer = umap.UMAP(n_neighbors=5,
                                min_dist=0.001,
                                metric='correlation', verbose=True).fit(self.zs.numpy())
        embedding = transformer.transform(self.zs.numpy())

        return embedding
    
    def reduce(self, epochs, hidden_size):
        self.create_dataloader()
        self.train_vae(epochs=epochs, hidden_size=hidden_size)
        self.run_vae()
        self.embedding = self.vae_umap()
        #self.plot_2d(self.embedding, self.attribute)

### 1.3 PlottingAgent

In [6]:
# not yet instantiated

# 2. BASIS API  example usage (for manual testing)

In [7]:
### Client loader
# dataholder = DataHolder("Glitne", [1300, 1502, 2], [1500, 2002, 2])
# dataholder.add_near('./data/3d_nearstack.sgy');
# dataholder.add_far('./data/3d_farstack.sgy');
# dataholder.add_horizon('./data/Top_Heimdal_subset.txt')
# dataholder.add_well('well_1', 36, 276//2)

In [8]:
### Processor
# processor1 = Processor(dataholder)

# Input1 = processor1([False, 12,52], normalise=True)
# Input2 = processor1([True, 12,52], normalise=True)

In [9]:
### UMAP RUN
# UMAP_a = UMAP(Input1)
# UMAP_a1 = UMAP_a.reduce(n_neighbors=50)

In [10]:
### VAE RUN
# VAE_a = VAE_model(Input1)

# run with different params
# VAE_a.create_dataloader()
# VAE_a.train_vae()
# VAE_a.run_vae()
# VAE_a.vae_umap()
# VAE_a.reduce()

# 3. WIDGET INTERFACE:

## Widget definitions:

In [11]:
# Widget Definitions

# Data Loading:
data_files_title = widgets.HTML(
    value="<b>File Pathnames:<b>",
)

near_text = widgets.Text(description='Near SEGY:', value='./data/3d_nearstack.sgy')
far_text = widgets.Text(description='Far SEGY:', value='./data/3d_farstack.sgy')
horizon_text = widgets.Text(description='Horizon .txt:', value='./data/Top_Heimdal_subset.txt')

inline_range_title = widgets.HTML(
    value="<b>In-line range:<b>",
)

xline_range_title = widgets.HTML(
    value="<b>X-line range:<b>",
)


inline_start = widgets.IntText(
    value=1300,
    description='Start:',
    width=0.05
)

inline_stop = widgets.IntText(
    value=1502,
    description='Stop:',
)

inline_step = widgets.IntText(
    value=2,
    description='Step:',
)

xline_start = widgets.IntText(
    value=1500,
    description='Start:',
)

xline_stop = widgets.IntText(
    value=2002,
    description='Stop:',
)

xline_step = widgets.IntText(
    value=2,
    description='Step:',
)

load_button = widgets.Button(
    description='Load Data',)

# Data processing:
flattening_title = widgets.HTML(
    value="<b>Horizon Flattening:<b>",
)

norm_title = widgets.HTML(
    value="<b>Normalisation:<b>",
)

flat_option = widgets.Dropdown(
    options=[True, False],
    value=True,
    description='True/False:',
    disabled=False,
)

above_add = widgets.IntText(
    value=12,
    description='Above Add:',)
    
below_add = widgets.IntText(
    value=52,
    description='Below Add:',)

norm_option = widgets.Dropdown(
    options=[True, False],
    value=True,
    description='True/False:',
    disabled=False,
)

process_button = widgets.Button(
    description='Process Input',)


# Model selection:
umap_title = widgets.HTML(
    value="<b>UMAP:<b>",
)

umap_neighbours = widgets.IntText(
    value=50,
    description='Neighbours:',)

umap_button = widgets.Button(
    description='Run Umap',)



vae_title = widgets.HTML(
    value="<b>VAE:<b>",
)

epoch_num = widgets.IntText(
    value=3,
    description='Epochs:',)

latent_dim = widgets.IntText(
    value=8,
    description='Latent size:',)

vae_button = widgets.Button(
    description='Run Vae',)


# Plotting:
plot_title = widgets.HTML(
    value="<b>Plot parameters:<b>",
)
plot_button = widgets.Button(
    description='Plot',)


attribute_toggle = widgets.ToggleButtons(
    options=['AVO ff', 'Horizon "Depth"'],
    description='Attribute:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular'],

)


# scrap:
out = widgets.Output(layout={'border': '1px solid black'})
clear_button = widgets.Button(description='Clear Output')

## Action/Logic Definitions:

In [12]:
# LOGIC

# Data load:
@load_button.on_click
def load_on_click(b):
    ### Client loader
    dataholder = DataHolder("Data", [inline_start.value, inline_stop.value, inline_step.value], 
                                    [xline_start.value, xline_stop.value, xline_step.value])
    dataholder.add_near(near_text.value);
    dataholder.add_far(far_text.value);
    dataholder.add_horizon(horizon_text.value)
    dataholder.add_well('well_1', 36, 276//2)
    
    # save to binary file
    with open("./pickled/data.pickle", "wb") as file_:
        pickle.dump(dataholder, file_, -1)
    file_.close()
    print('Data Loaded successfully')

# Data process:
@process_button.on_click
def process_on_click(b):
    # load data
    file_pi2 = open('./pickled/data.pickle', 'rb')
    dataholder = pickle.load(file_pi2)
    file_pi2.close()
    
    #processing
    processor = Processor(dataholder)
    input1 = processor([flat_option.value, above_add.value, below_add.value], norm_option.value)
    
    # save to binary file
    with open("./pickled/input.pickle", "wb") as file_:
        pickle.dump(input1, file_, -1)
    file_.close()
    print('Data Input processed successfully')

# Run Umap
@umap_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()
    # UMAP RUN
    UMAP_a = UMAP(input1)
    UMAP_a.reduce(umap_neighbours.value)
    
    # save to binary file
    with open("./pickled/model.pickle", "wb") as file_:
        pickle.dump(UMAP_a, file_, -1)
    file_.close()
    
# Run VAE
@vae_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()
    
    # VAE RUN
    VAE_a = VAE_model(input1)
    VAE_a.reduce(epochs=epoch_num.value, hidden_size=latent_dim.value)
    
     # save to binary file
    with open("./pickled/model.pickle", "wb") as file_:
        pickle.dump(VAE_a, file_, -1)
    file_.close()
    
#Plot
@plot_button.on_click
def plot_on_click(b):
    # load data holder
    file1 = open('./pickled/data.pickle', 'rb')
    dataholder = pickle.load(file1)
    file1.close()
    
    # load embedding
    file2 = open('./pickled/model.pickle', 'rb')
    model = pickle.load(file2)
    file2.close()
    
    if attribute_toggle.value == 'Horizon "Depth"':
        print('Horizon attribute')
        attr = dataholder.horizon.reshape(25351)
        
    elif attribute_toggle.value == 'AVO ff':
        print('AVO attribute')
        attr = np.min(model.FF, axis=(1))
        
    else: print('error attr input not recognised')
    
    with out:
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        sc = ax.scatter(model.embedding[::, 0], model.embedding[::, 1], s=2.0, c=attr)
        print('plotted')

@clear_button.on_click
def clear_on_click(b):
    out.clear_output()

## GUI/Interface Layout Structuring

In [13]:
# GUI LAYOUT
tab1 = VBox(children=[HBox(children=[VBox(children=[data_files_title, near_text, far_text, horizon_text]), 
                                     VBox(children=[inline_range_title, inline_start, inline_stop, inline_step]),
                                     VBox(children=[xline_range_title, xline_start, xline_stop, xline_step]),
                                     ]),
                      load_button  
                     ])
                     

tab2 = VBox(children=[HBox(children=[VBox(children=[flattening_title, flat_option, above_add, below_add,]),
                                     VBox(children=[norm_title, norm_option])
                                    ]),     
                      process_button
                     ])

tab3 = VBox(children=[HBox(children=[VBox(children=[umap_title, umap_neighbours, umap_button]),
                                     VBox(children=[vae_title, epoch_num, latent_dim, vae_button]),
                                    ])
                     ])

tab4 = VBox(children=[HBox(children=[VBox(children=[plot_title, attribute_toggle, plot_button]),
                                     VBox(children=[]),
                                    ])
                     ])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, '1. Data Loading')
tab.set_title(1, '2. Data Processing')
tab.set_title(2, '3. Model Selection')
tab.set_title(3, '4. Visualisation')

VBox(children=[tab, clear_button, out])